In [ ]:
def retention(data, start_date, end_date, periods, cohort_type='monthly_cohorts', cohort_intervals='month'):
    '''
    Функция для подсчета retention
    
        Параметры:
            data: Датафрейм с данными, где
                auth_ts - Дата авторизации в приложении в формате timestamp
                reg_ts - Дата регистрации в приложении в формате timestamp
                start_date: Начало периода
                end_date: Конец периода
                cohort_type: Тип когорты(по дням/неделям/месяцам)
                cohort_intervals: Интервалы(дни/недели/месяцы)
        
        Возвращает таблицу с retention для выбраного типа когорты и интервала
             
    '''
    
    data.auth_ts = pd.to_datetime(data.auth_ts, unit='s')
    data.reg_ts = pd.to_datetime(data.reg_ts, unit='s') 
    
    if cohort_intervals == 'month':
        data['month'] = (data.auth_ts - data.reg_ts).div(np.timedelta64(1, 'M')).astype(int)        
    elif cohort_intervals == 'week':
        data['week'] = (data.auth_ts - data.reg_ts).dt.days.div(7).astype(int)         
    elif cohort_intervals == 'day':
        data['day'] = (data.auth_ts - data.reg_ts).dt.days
     
    if cohort_type == 'monthly_cohorts':
        data['monthly_cohorts'] = data.reg_ts.dt.to_period('M')
    elif cohort_type == 'weekly_cohorts':     
        data['weekly_cohorts'] = data.reg_ts.dt.to_period('W')
    elif cohort_type == 'dayly_cohorts':
        data['dayly_cohorts'] = data.reg_ts.dt.to_period('d')
    
    grouped = data.query('reg_ts >=  @start_date &  reg_ts <= @end_date ')\
                  .groupby([cohort_type, cohort_intervals], as_index=False)\
                  .agg({'uid': 'nunique'})
    
    t_pivot = grouped.pivot(index=cohort_type, columns=cohort_intervals, values='uid')
    t_pivot = t_pivot[periods]
    return round(t_pivot.div(t_pivot[0].values, axis=0), 3)